<a href="https://colab.research.google.com/github/kalopez0621/NLP_Projects/blob/main/LlamaTutorApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Purpose:

**LlamaTutor** is an AI-powered educational platform designed to help middle school students, specifically those in Florida, with math topics aligned to Florida's B.E.S.T. Standards. It offers personalized tutoring by leveraging state-of-the-art natural language models (like Llama 3.1) and Together AI's API. The application provides students with:

* Interactive explanations of key concepts.
* Step-by-step problem-solving.
* Practice questions.
* Sources from curated materials like Florida's official benchmarks.


#  How LlamaTutor Works

**User Input:**

* Students can type in a topic or select a predefined subject (e.g., Number Sense, Algebraic Reasoning).
* The application also accepts specific queries like "Teach me how to multiply fractions."
**Integration of Florida Standards:**

* The system utilizes official educational benchmarks stored in JSON format. These benchmarks are extracted from PDFs provided by the Florida Department of Education.
* Example: When a student asks about "Number Sense," the application refers to the stored benchmarks for Grade 6.
**Response Generation:**
* LlamaTutor uses Together AI's API to fetch responses based on user input.
* Example Workflow:
  * A student's question is sent to Together AI's chat/completions endpoint.
  * The API returns a response, which is displayed to the user in a conversational format.
**Educational Sources:**
* In addition to generating answers, LlamaTutor fetches reliable sources (web pages, PDFs) related to the student's query using integrated search APIs (e.g., Serper or Bing).
* These sources provide additional reading material or videos for further learning.
**Interactive Design:**
*The app includes quizzes, exercises, and interactive follow-up questions to test comprehension.
* Example: After explaining how to multiply fractions, the app asks, "What is the product of 2/3 and 3/4?"
**Deployment and Accessibility:**
* The application is fully deployed on Vercel, allowing easy access for students and teachers. However, this notebook demonstrates its core functionality within Colab.


Step 1: Setup and Installation

In [1]:
# Install necessary libraries
!pip install pdfplumber openai pandas requests


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.1 MB/s eta 0:00:00


Step 2: Florida Standards Integrations
Loading Florida standards(benchmarks) from a JSON file

In [2]:
import json

# Load Florida standards
def load_standards():
    # Simulating the standards JSON file from GitHub
    # Replace this URL with your actual JSON file hosted on GitHub
    json_url = "https://raw.githubusercontent.com/kalopez0621/LlamaTutor_New/main/public/standards/grade6_benchmarks.json"
    try:
        import requests
        response = requests.get(json_url)
        if response.status_code == 200:
            standards = response.json()
            return standards
        else:
            print(f"Error loading standards: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

standards = load_standards()
if standards:
    print("Standards successfully loaded!")
    print(standards["benchmarks"][:5])  # Display first 5 benchmarks
else:
    print("Failed to load standards.")


Standards successfully loaded!
['MA.6.NSO.1.1 Page 2', 'MA.6.NSO.1.2 Page 3', 'MA.6.NSO.1.3 Page 4', 'MA.6.NSO.1.4 Page 4', 'MA.6.NSO.2.1 Page 5']


Step 3: API Calls and Response Handling
LlamaTutor uses Together AI's API for generating responses

In [3]:
import requests

# Together AI API Call
TOGETHER_API_KEY = "f67c747c91109cd9fb2c7327b4107ff5a43dc49db4281cc567f1a0e432ab512a"  # Replace with your API key

def get_response(question):
    url = "https://together.helicone.ai/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {TOGETHER_API_KEY}"
    }
    payload = {
        "model": "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        "messages": [{"role": "user", "content": question}],
        "stream": False
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return data["choices"][0]["message"]["content"]
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return "Sorry, I couldn't generate a response."
    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred."

# Test API call
question = "Can you teach me how to multiply fractions?"
response = get_response(question)
print(response)


Multiplying fractions is a relatively simple process. Here's a step-by-step guide:

**Multiplying Fractions:**

1. **Write down the fractions**: Start by writing down the fractions you want to multiply. For example, let's say you want to multiply 1/2 and 3/4.

   1/2 × 3/4

2. **Multiply the numerators**: Multiply the top numbers (the numerators) of the fractions. In this case, you multiply 1 and 3.

   1 × 3 = 3

3. **Multiply the denominators**: Multiply the bottom numbers (the denominators) of the fractions. In this case, you multiply 2 and 4.

   2 × 4 = 8

4. **Write the product**: Write the product of the numerators (3) over the product of the denominators (8).

   3/8

That's it! The result of multiplying 1/2 and 3/4 is 3/8.

**Example 2:**

Let's say you want to multiply 2/3 and 5/6.

2/3 × 5/6

1. Multiply the numerators: 2 × 5 = 10
2. Multiply the denominators: 3 × 6 = 18
3. Write the product: 10/18

To simplify the fraction, you can divide both the numerator and the denomina

Step 4: Interactive Tutoring Workflow
Simulates how the application interacts with students.

In [4]:
def interactive_tutor():
    print("Welcome to LlamaTutor!")
    while True:
        topic = input("Enter a topic you'd like to learn (or 'exit' to quit): ")
        if topic.lower() == "exit":
            print("Thank you for using LlamaTutor!")
            break

        # Check if topic exists in Florida standards
        related_benchmarks = [
            bm for bm in standards["benchmarks"] if topic.lower() in bm.lower()
        ]
        if related_benchmarks:
            print(f"Related Benchmarks: {related_benchmarks[:3]}")
        else:
            print("No related benchmarks found.")

        # Get response from Together AI
        response = get_response(topic)
        print("\nLlamaTutor Response:")
        print(response)

interactive_tutor()


Welcome to LlamaTutor!
Enter a topic you'd like to learn (or 'exit' to quit): Factions
No related benchmarks found.

LlamaTutor Response:
Factions can refer to various groups or organizations within a society, community, or system, often with distinct interests, values, or goals. Here are some examples of factions:

1. **Political Factions**: Groups within a political party or government that have different ideologies, policies, or leadership styles. Examples include the Democratic and Republican parties in the United States.
2. **Social Factions**: Groups within a society that have distinct social, cultural, or economic characteristics. Examples include racial or ethnic groups, social classes, or subcultures.
3. **Military Factions**: Groups within a military organization that have different loyalties, ideologies, or goals. Examples include rebel groups, insurgent organizations, or rival factions within a military.
4. **Business Factions**: Groups within a company or industry that hav

# Full Project

The complete LlamaTutor project, including deployment, is available on [GitHub](https://github.com/kalopez0621/LlamaTutor_New).


# Production Deployment with Vercel

To ensure the LlamaTutor application is easily accessible to users, I deployed it using Vercel, a cloud platform for hosting and scaling front-end applications. The deployment process allows the application to run seamlessly online with minimal setup, making it accessible to middle school students, parents, and educators through a simple URL.

**Deployment Process:**
- Repository Hosting: The LlamaTutor codebase is hosted on GitHub in a repository titled LlamaTutor_New. This repository contains all the files and configurations needed for deployment.
- Integration with Vercel: The repository was linked to a Vercel account, and the deployment was initiated with a single click.
- Continuous Deployment: Vercel monitors the repository for updates, automatically rebuilding and redeploying the application whenever changes are pushed to the main branch. This ensures that the application remains up-to-date with the latest features and fixes.

**Features of the Deployment:**
- Public Accessibility: The application is hosted on a custom URL provided by Vercel, making it easy for anyone to access without needing to install additional software.
- Scalability: Vercel ensures the application can handle multiple users simultaneously without degradation in performance.
- Global Distribution: With Vercel’s global content delivery network (CDN), the application loads quickly for users anywhere in the world.
- Custom Domain: Optionally, a custom domain can be configured to further personalize the user experience.
Production URL:

The deployed version of the LlamaTutor can be accessed at the following URL:
[LlamaTutor Deployed App](https://https://llama-tutor-new.vercel.app/)

# **Business Use Case**

LlamaTutor is an interactive AI-driven tutoring platform designed to bridge the gap in personalized learning for middle school students in Florida. Traditional classroom settings often struggle to address the unique pace and understanding of every student, leading to gaps in comprehension and engagement. LlamaTutor leverages cutting-edge AI models, like Together AI and Llama 3.1, to provide an intuitive, interactive learning experience tailored specifically to Florida's educational standards. By integrating the Florida Benchmarks for Excellent Student Thinking (B.E.S.T.), LlamaTutor ensures that students receive support and exercises aligned with their curriculum, addressing common challenges in mathematics, such as algebraic reasoning, geometric reasoning, and data analysis.

Through conversational interactions, LlamaTutor not only explains concepts but also quizzes students to reinforce understanding, ensuring they master each topic effectively. The application eliminates the need for expensive one-on-one tutoring by offering on-demand assistance for topics students struggle with, enhancing their confidence and performance in state tests and beyond.

# **Business Value**

LlamaTutor delivers significant value to schools, parents, and educational institutions by promoting equity in learning. Schools can integrate the platform to complement classroom instruction, especially for students requiring additional support, thus boosting overall academic outcomes. For parents, the platform serves as an affordable alternative to private tutoring, providing high-quality, curriculum-aligned guidance at no extra cost.

Educational institutions can also benefit from data-driven insights into student performance, as LlamaTutor has the potential to track common areas of difficulty, enabling teachers to refine their instructional strategies. By aligning with Florida's educational standards, the application ensures compliance with curriculum requirements, making it a reliable tool for academic success. The automation and scalability of the AI-driven tutor reduce operational costs for schools while maintaining high-quality educational support for every student, empowering them to achieve excellence in their studies.